In [5]:
import torch
import schnetpack as spk
import math
from schnetpack.datasets import QM9


import numpy as np
from numpy import savetxt

# Define Important Functions

In [6]:
def print_molecule(props):
    
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 
    print('')  
    return x,y,z

def hook_v0(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v0
    #Update the embedding_output variable to be equal to our output tensor
    v0=out_tensor 

def hook_v1(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v1
    #Update the embedding_output variable to be equal to our output tensor
    v1=out_tensor 

def hook_v2(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v2
    #Update the embedding_output variable to be equal to our output tensor
    v2=out_tensor  
    
def hook_emb(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global emb
    #Update the embedding_output variable to be equal to our output tensor
    emb=out_tensor 
#def convert_2D(number_of_atoms,rep):
#    layer = np.zeros((number_of_atoms,30))
#    for i in range(number_of_atoms):
#        for j in range(30):
#            layer[i][j] = rep[0][i][j]
#    return layer



In [7]:
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)
checkpoint_path = '../../../../data/trained_models/qm9_i3_30f_10000_5000/trained.pth'
split_file='../../../../data/trained_models/qm9_i3_30f_10000_5000/split.npz'
number_of_inputs = 5000

# Load split file 
train, val, test = spk.data.train_test_split(qm9data,split_file=split_file)

# Load atom ref data 
atomrefs = qm9data.get_atomref(QM9.U0)
print('U0 of hyrogen:', '{:.2f}'.format(atomrefs[QM9.U0][1][0]), 'eV')
print('U0 of carbon:', '{:.2f}'.format(atomrefs[QM9.U0][6][0]), 'eV')
print('U0 of oxygen:', '{:.2f}'.format(atomrefs[QM9.U0][8][0]), 'eV')
print('U0 of oxygen:', '{:.2f}'.format(atomrefs[QM9.U0][7][0]), 'eV')   

# Define SchNet representation model

schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=3,
    cutoff=4. , cutoff_network=spk.nn.cutoff.CosineCutoff
)

# Define SchNet output model and property to be predicted

output_U0 = spk.atomistic.Atomwise(n_in=30, atomref=atomrefs[QM9.U0])

# Define atomistic model

model = spk.AtomisticModel(representation=schnet,output_modules=output_U0)

# Load saved checkpoint file
load_checkpoint = torch.load(checkpoint_path)

#qm9_i6_30f_20g-1000-500-4_300.pth
# load model's state dictionary from saved checkpoint
model.load_state_dict(load_checkpoint)

#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

datao = np.zeros((1,30))
datahae = np.zeros((1))
dataoae = np.zeros((1))
datah = np.zeros((1,30))
data = np.zeros((1,4))

print(data)
for idx in range(number_of_inputs):

    # load data for molecule
    at, props = qm9data.get_properties(idx)

    
    # print molecule for identification
    print(idx)
    x, y, z = print_molecule(props)
    number_of_atoms=len(z)

    # convert qm9 data to machine-readable form
    inputs = converter(at)

    #Instatiate layer output
    v0=None
    v1=None
    v2=None
    x=None       

    # Forward hook the model's interaction layer 
    model.representation.interactions[0].register_forward_hook(hook_v0)

    # Forward hook the model's interaction layer 
    model.representation.interactions[1].register_forward_hook(hook_v1)

    # Forward hook the model's interaction layer 
    model.representation.interactions[2].register_forward_hook(hook_v2)

    # Forward hook the model's interaction layer 
    model.representation.embedding.register_forward_hook(hook_emb)

    # Forward pass molecules through the model
    model(inputs)


    rep = emb + v0 + v1 + v2
    rows = np.zeros((number_of_atoms,30))
    for i in range(number_of_atoms):
        for j in range(30):
            rows[i][j] = rep[0][i][j]

    from schnetpack.atomistic.output_modules import yi

    yi=yi.detach().numpy()

    #save the vector of every oxygen atom encountered
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            datao = np.vstack((datao,rows[i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datah = np.vstack((datah,rows[i]))    
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datahae = np.vstack((datahae,yi[0][i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            dataoae = np.vstack((dataoae,yi[0][i]))
            

    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2] 
    
    distance = np.zeros((number_of_atoms))
    dist_x = np.zeros((number_of_atoms))
    dist_y = np.zeros((number_of_atoms))    
    dist_z = np.zeros((number_of_atoms))

    #calculate E distance, to figure out neighbor
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            for j in range(number_of_atoms):
                dist_x[j] = x[i] - x[j]
                dist_y[j] = y[i] - y[j]
                dist_z[j] = z[i] - z[j]
                distance[j] = math.sqrt(dist_x[j]**2 + dist_y[j]**2 + dist_z[j]**2)
            
            neighbor_dist = distance[0]                    
            neighbor = 0
            for k in range(number_of_atoms):
                if distance[k] < neighbor_dist and distance[k] != 0:
                    neighbor_dist = distance[k]
                    neighbor = k
                
            
            neighbor2_dist = neighbor_dist                    
            neighbor2 = 0
            for k in range(number_of_atoms):
                if distance[k] < neighbor2_dist and distance[k] != 0 and props['_atomic_numbers'][k] != 1:
                    neighbor2_dist = distance[k]
                    neighbor2 = k
                    
            neighbor3_dist = neighbor2_dist                    
            neighbor3 = 0
            for k in range(number_of_atoms):
                if distance[k] < neighbor3_dist and distance[k] != 0 and props['_atomic_numbers'][k] != 1:
                    neighbor3_dist = distance[k]
                    neighbor3 = k
            
            neighbor4_dist = neighbor3_dist                    
            neighbor4 = 0
            for k in range(number_of_atoms):
                if distance[k] < neighbor4_dist and distance[k] != 0 and props['_atomic_numbers'][k] != 1:
                    neighbor4_dist = distance[k]
                    neighbor4 = k
            
        
            first = float((1/(neighbor_dist))*yi[0][neighbor])
            second = float((1/(2*neighbor2_dist))*yi[0][neighbor2])
            third = float((1/(3*neighbor3_dist))*yi[0][neighbor3])
            fourth = float((1/(4*neighbor4_dist))*yi[0][neighbor4])
            row = (first,second,third,fourth)
            data = np.vstack((data, row))
print(data)

U0 of hyrogen: -13.61 eV
U0 of carbon: -1029.86 eV
U0 of oxygen: -2042.61 eV
U0 of oxygen: -1485.30 eV
[[0. 0. 0. 0.]]
0
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613

1
N -0.03160822 0.021159047 0.060332563
H 0.026075294 -0.9904034 -0.029608395
H 0.9246072 0.35579658 -0.030988995
H -0.5114599 0.34058344 -0.77777386

2
O -0.056248687 0.035458494 0.0007260183
H 0.042878296 -0.9215089 -0.00534094
H 0.84990215 0.35871136 -0.0061824406

3
C 0.5995395 0.0 0.0
C -0.5995395 0.0 0.0
H -1.6616386 0.0 0.0
H 1.6616386 0.0 0.0

4
C -0.0075630867 0.5570853 0.003070365
N 0.008071566 -0.59453905 -0.0032767907
H -0.022041854 1.6235689 0.008948269

5
C -0.008137181 0.5992856 0.002790173
O 0.0081545 -0.60058945 -0.0028012523
H 0.9208008 1.2085854 -0.0010057696
H -0.9532692 1.1830921 0.012220519

6
C -0.010403622 0.7647314 0.004216852
C 0.010404125 -0.7647325 -0

O 0.9053031 1.4252946 0.50014365
C 0.7614709 0.06763947 0.1408983
C -0.6973357 -0.34277213 -0.01985505
O -1.0657501 -1.318754 -0.6174562
H 0.6943127 1.966276 -0.2672169
H 1.3159493 -0.211613 -0.76589656
H 1.1789486 -0.52163213 0.9679234
H -1.406806 0.35434428 0.48486584

38
C -0.39021975 1.8088213 0.65719855
C -0.35307074 0.27912098 0.62026554
C 0.3530644 -0.2791212 -0.62026495
C 0.39022544 -1.8088214 -0.6571987
H 0.62180984 2.2292233 0.6553988
H -0.9159678 2.2128956 -0.21531074
H -0.9009811 2.1776998 1.5523278
H 0.14914319 -0.09831827 1.521004
H -1.3774784 -0.11465078 0.6594648
H -0.14915404 0.098314516 -1.521002
H 1.3774698 0.114655964 -0.65946597
H 0.9160251 -2.2128918 0.21528102
H -0.6218005 -2.2292323 -0.65534467
H 0.90094143 -2.1776922 -1.5523572

39
C 0.74676186 1.8234588 0.0026987367
C 0.71465826 0.29370835 -0.0053245095
C -0.7016382 -0.2650462 0.0052626343
O -0.61744595 -1.6824552 -0.003045157
H 1.7753277 2.1956031 -0.005183291
H 0.23768522 2.239137 -0.8741201
H 0.25471118 2.2

C -0.024941536 1.5804 0.6270696
C -0.005323342 0.045704123 0.5899224
O -1.3089012 -0.5031127 0.7618272
C 0.63353485 -0.45244792 -0.6337418
C 1.121628 -0.8410348 -1.661492
H 0.9908436 1.9844332 0.5898606
H -0.5801212 1.9766587 -0.22921759
H -0.5106804 1.9140329 1.5476044
H 0.5607209 -0.3199105 1.4551519
H -1.802564 -0.33429405 -0.048530698
H 1.56339 -1.198912 -2.5586312

64
C 0.5316238 1.5948187 -0.28219584
C 0.54895324 0.06251989 -0.23920514
O 1.2624056 -0.45100507 0.86912805
C -0.82840264 -0.47825956 -0.28057352
N -1.9092673 -0.8870317 -0.28234872
H 1.5604715 1.9616041 -0.26227635
H 0.038933463 1.9549932 -1.1894974
H -0.0064558904 1.9990758 0.5810188
H 1.0775769 -0.31964225 -1.1192273
H 0.81857485 -0.16114564 1.6746999

65
C 0.48407435 1.6903256 -0.08229043
C 0.5250599 0.15250751 -0.07279259
C 1.2577494 -0.38233417 1.1697518
C -0.89174175 -0.38703817 -0.090031154
O -1.3658336 -1.084269 -0.94844854
H 1.4986078 2.0978053 -0.03739774
H 0.003119928 2.0730886 -0.98730856
H -0.0671943 2.075

H -0.2978252 1.9987627 -1.0714314
H -0.98947597 1.9727021 0.5610634
H -1.4028863 -0.31910384 -1.4131526
H 1.1203744 -1.533271 -0.9140839
H 2.0860763 -0.032088093 -0.5136909
H 1.0760279 -0.05574824 1.8361567
H 0.12799071 -1.5535523 1.3875387

85
N -0.057123955 1.7483617 0.49261686
C -0.051818028 0.54126346 0.15909214
C -1.1001686 -0.5495181 0.060917392
C 0.08602445 -1.4154221 -0.40646124
O 0.9838027 -0.27093995 -0.2644824
H 0.87668735 2.15694 0.42496663
H -1.5559136 -0.8302318 1.0115194
H -1.8817728 -0.36501205 -0.6774614
H 0.38472512 -2.2301471 0.25738493
H 0.05677837 -1.7620572 -1.4421631

86
O 0.018968131 -1.7640274 -0.34606293
C 0.00592748 -0.58591306 -0.14286245
C -1.1154436 0.44729778 0.07466429
C -0.015023246 1.5084827 0.3856067
C 1.1046822 0.47698423 0.045027785
H -1.6964426 0.6352329 -0.8339263
H -1.8139421 0.20165703 0.8795269
H -0.03536181 2.3933592 -0.25162515
H -0.0054291748 1.831497 1.4278737
H 1.8307295 0.2503917 0.8308742
H 1.6559936 0.68006015 -0.87867796

87
O 0.056591

N 0.7839506 2.1481187 0.004280184
C 0.85980904 0.89550203 -0.0035450475
O -0.297446 0.1635308 0.003514263
C -0.19885314 -1.1969259 -0.0045071156
O -1.15787 -1.8988776 0.00095024036
H 1.7159578 2.5592022 -0.0023584922
H 1.766076 0.27907202 -0.015709855
H 0.84739363 -1.5530415 -0.016322123

110
O -1.4572579 -0.8431472 0.008857734
C -1.2600759 0.33831686 0.012946789
N -0.013109395 0.96676713 0.005070856
C 1.2504364 0.3724739 -0.009277258
O 1.4795827 -0.80318785 -0.017458055
H -2.068963 1.0969422 0.02400012
H -0.026811669 1.9773167 0.010396228
H 2.038459 1.1528236 -0.012080559

111
C -0.03515972 2.584621 0.01434152
C -0.015212007 1.133484 0.006053129
C 0.0012401405 -0.07611664 -0.0005787047
C 0.019471643 -1.4368443 -0.00795138
C 0.035693187 -2.644352 -0.014370795
H 0.98127615 2.9938614 0.0069270316
H -0.55998194 2.9776597 -0.8635865
H -0.5430845 2.9676716 0.9064997
H 0.0499 -3.7057338 -0.019976754

112
C -0.03527519 2.5978003 0.0143258525
C -0.015304027 1.148424 0.0061399327
C 0.0011212354

...

4925
C -0.15910433 2.1603673 0.09650369
C -0.032331914 0.6725595 0.034586795
C 0.008105289 -0.1065303 -1.0825175
N -0.024045171 0.2440697 -2.4361548
O 0.08177063 -1.4274592 -0.7528769
C 0.09454946 -1.4944875 0.619638
C 0.034126684 -0.24146378 1.1487498
N -0.06552526 0.14431235 2.4948545
H 0.7630618 2.6469123 0.44218332
H -0.38963225 2.5583484 -0.8950755
H -0.95767695 2.461579 0.78447247
H 0.9043217 0.35241848 -2.8341653
H -0.5148324 -0.45637512 -2.9816866
H 0.112099364 -2.4825637 1.0473547
H 0.19909038 -0.59344065 3.1350358
H 0.48160788 0.97107345 2.7012413

4926
C 0.11672464 2.1675427 0.12290583
C 0.026480699 0.6780875 0.034101605
C -0.05934113 -0.083888315 -1.093466
N -0.10716931 0.29023233 -2.4392118
O -0.09289294 -1.4100162 -0.78143376
C -0.022511546 -1.5092739 0.5864697
C 0.038153123 -0.26283422 1.1233802
O 0.11902407 0.023778833 2.4517355
H 0.95379853 2.4875417 0.75733316
H 0.276538 2.5897734 -0.8723784
H -0.7946571 2.621138 0.5339917
H -1.0573037 0.38857308 -2.784905
H 0.3676401

C 0.031676907 2.5510612 -0.523985
C -0.045002688 1.057004 -0.5307952
C -0.28153628 0.2216684 -1.5774578
O -0.2740407 -1.0759959 -1.1836648
C -0.024751676 -1.098458 0.16573071
C 0.03561442 -2.4136508 0.849042
C 0.12080185 0.18357693 0.6010447
O 0.3786543 0.5427365 1.8930979
H 1.0125195 2.9100761 -0.18602094
H -0.13363488 2.9567454 -1.5254191
H -0.7233844 2.9976377 0.1360234
H -0.4668237 0.39821297 -2.6247263
H 0.8274121 -3.0522637 0.43936765
H 0.24051373 -2.2552958 1.9104345
H -0.90848297 -2.964652 0.76140195
H 0.43605915 1.5012872 1.9401402

4942
C -0.06506382 2.5510118 -0.5459295
C -0.03308727 1.0553513 -0.5273578
C -0.04387632 0.22195178 -1.594969
O -0.005723335 -1.0900048 -1.1815568
C 0.022064399 -1.0677947 0.17153141
N 0.13913392 -2.3145134 0.7793572
C 0.0042557456 0.21057147 0.6491784
C 0.023029026 0.65551114 2.078625
H 0.82291365 2.9806812 -0.066896
H -0.108073555 2.932372 -1.5699309
H -0.93663985 2.9436834 -0.008148111
H -0.07849304 0.37318444 -2.6604636
H -0.40617442 -3.0247679

C -0.10601474 2.049836 -0.008482629
N 0.0056029605 0.6083808 0.007574542
C 0.013305128 -0.16168693 -1.1419965
O -0.010185998 0.20322987 -2.2891538
O 0.047700334 -1.4673363 -0.73000115
C 0.07456356 -1.4895569 0.66058326
C 0.03500531 -0.23031834 1.1250342
N -0.08006933 0.30709052 2.4067848
H 0.8284342 2.5345886 0.29809186
H -0.32792795 2.3411598 -1.0362581
H -0.91130286 2.3777747 0.65447545
H 0.100080766 -2.4522498 1.1368619
H -0.11947904 -0.41672048 3.1130497
H 0.6686493 0.95304155 2.636241

4960
C 0.08633795 1.98557 0.009720658
N -0.011084618 0.54178697 0.0029921632
C -0.01983015 -0.25060982 -1.1321397
C -0.016102614 0.3207907 -2.5118797
C -0.034288745 -1.5627544 -0.7187417
C -0.026525753 -1.5703449 0.7023375
C -0.0071332753 -0.2627082 1.1296242
C 0.012203727 0.29342228 2.5154793
H 1.1288484 2.3295977 -0.00073485606
H -0.4187378 2.3995116 -0.8654623
H -0.39766273 2.3894126 0.9012669
H 0.8672029 0.9407082 -2.7132409
H -0.016445046 -0.49450076 -3.239129
H -0.89853346 0.9414423 -2.717276


4975
C -0.07938918 2.4183617 -0.55560386
N -0.039555926 0.97414404 -0.5541288
C -0.04283521 0.18132277 -1.668698
C 0.0026884389 -1.1375164 -1.2717712
C 0.03455677 -1.1275953 0.14780445
O 0.08349392 -2.19789 1.0021975
C 0.008154322 0.18046768 0.58305305
C 0.024603637 0.72351223 1.9709964
H 0.80846304 2.8437574 -0.07449376
H -0.11316079 2.769786 -1.5884407
H -0.96597975 2.7947853 -0.033156563
H -0.07726018 0.6185593 -2.6540647
H 0.011881181 -2.0008802 -1.9205353
H 0.09415676 -3.002145 0.4770884
H 0.8956802 1.3625175 2.1683834
H -0.86973625 1.3152441 2.2077558
H 0.06265204 -0.111393616 2.6743019

4976
C -0.047236707 2.4146693 -0.58563215
N 0.023374934 0.96963596 -0.57364947
C 0.09003997 0.16219224 -1.6719477
C 0.08879711 -1.1554763 -1.258208
C 0.007139113 -1.1333554 0.15686652
O -0.017078154 -2.184618 1.0364243
C -0.036637705 0.18243451 0.5699607
N -0.07618852 0.7682814 1.8538043
H 0.6479942 2.8160632 -1.3279524
H -1.0571696 2.7715895 -0.8220611
H 0.22698241 2.7788427 0.40517828
H 0.12616

4991
C -1.6072196 -2.7363586 0.32084158
C -1.5627209 -1.5588669 0.09103047
C -1.5146264 -0.12549594 -0.1865949
C -0.12857234 0.44242677 -0.18531172
N 0.46166945 1.0917605 -1.1372157
C 1.7100329 1.4125124 -0.6146701
C 1.7995381 0.9264718 0.6436163
O 0.6199322 0.29943463 0.9355738
H -1.6442505 -3.7782733 0.52225786
H -1.940127 0.09082243 -1.1710529
H -2.1280909 0.41020593 0.5505391
H 2.4408393 1.9631058 -1.1848828
H 2.5496616 0.92407453 1.4151214

4992
N -1.7283653 -2.4189668 -0.9425321
C -1.63491 -1.4594806 -0.3052465
C -1.4880472 -0.21250962 0.45058718
C -0.10150284 0.36028612 0.36102188
C 0.63706166 1.0532607 1.2931865
C 1.8627894 1.426874 0.675059
C 1.835702 0.94621956 -0.61261266
N 0.63551 0.308172 -0.796648
H -2.2470076 0.48996603 0.077164784
H -1.7342023 -0.40257478 1.5000259
H 0.32713002 1.271541 2.3040438
H 2.6703515 1.9812268 1.1272098
H 2.5646052 0.99336374 -1.4052695
H 0.36583632 -0.19975375 -1.6227493

4993
N -1.6008111 -2.6675131 0.43672565
C -1.5692867 -1.5378246 0.1987794

In [8]:
print('DONE')
savetxt('../../../../data/data.csv',data,delimiter=',') 
savetxt('../../../data/dataO.csv',datao,delimiter=',') 
savetxt('../../../data/dataH.csv',datah,delimiter=',') 
savetxt('../../../data/hae.csv',datahae,delimiter=',') 
savetxt('../../../data/oae.csv',dataoae,delimiter=',')
        

DONE


FileNotFoundError: [Errno 2] No such file or directory: '../../../data/dataO.csv'